# Working With Country Population Data From The CIA World Factbook Database

This project uses SQLite aggregate functions and subqueries to extract data based on questions regarding the following country attributes;

* Population
* Population Growth
* Land area
* Water area
* Birth rate
* Death Rate

## Load The Database And Return Table Information

In [13]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [14]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Show The Top 5 Rows

In [15]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Find The Minimum And Maximum Values For Population And Population Growth

In [16]:
%%sql
SELECT MIN(population), MAX(population), 
MIN(population_growth), MAX(population_growth)
  FROM facts; 

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


## Find The Countries With The Lowest And Highest Population

In [17]:
%%sql
SELECT name, MIN(population)
FROM facts;

Done.


name,MIN(population)
Antarctica,0


In [18]:
%%sql
SELECT name, MAX(population)
  FROM facts; 

Done.


name,MAX(population)
World,7256490011


## Exclude 'World' From The Search

In [19]:
%%sql
SELECT MIN(population), MAX(population), 
MIN(population_growth), MAX(population_growth)
  FROM facts
WHERE name != 'World';

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


In [20]:
%%sql
SELECT name, MAX(population)
FROM facts
WHERE name <> 'World';

Done.


name,MAX(population)
China,1367485388


## Find Average Population And Average Area

In [21]:
%%sql
SELECT AVG(population), AVG(area)
FROM facts;

Done.


AVG(population),AVG(area)
62094928.32231405,555093.546184739


## Find Countries Where Population Is Higher Than The Average And Area Is Less Than The Average

In [22]:
%%sql
SELECT name, population, area
FROM facts
WHERE population > (SELECT AVG(population)
                    FROM facts)
AND area < (SELECT AVG(area)
            FROM facts);

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Japan,126919659,377915
Philippines,100998376,300000
Thailand,67976405,513120
United Kingdom,64088222,243610
Vietnam,94348835,331210


## Find The Maximum Population Growth

In [23]:
%%sql
SELECT name, MAX(population_growth)
FROM facts;

Done.


name,MAX(population_growth)
South Sudan,4.02


## Find The Countries With The Largest Water Area

In [24]:
%%sql
SELECT name, ROUND(CAST(area_water AS Float) / CAST(area_land AS Float), 6) AS ratio_water_to_land
FROM facts
ORDER BY ratio_water_to_land DESC
LIMIT 20;

Done.


name,ratio_water_to_land
British Indian Ocean Territory,905.666667
Virgin Islands,4.520231
Puerto Rico,0.554791
"Bahamas, The",0.386613
Guinea-Bissau,0.284673
Malawi,0.259396
Netherlands,0.22571
Uganda,0.222922
Eritrea,0.164356
Liberia,0.15624


In [25]:
%%sql
SELECT name
FROM facts
WHERE area_water > area_land;

Done.


name
British Indian Ocean Territory
Virgin Islands


## Find The Population Increase

In [26]:
%%sql
SELECT name, birth_rate - death_rate AS pop_increase
FROM facts
ORDER BY pop_increase DESC
LIMIT 20;

Done.


name,pop_increase
Malawi,33.150000000000006
Uganda,33.1
Niger,33.03
Burundi,32.739999999999995
Mali,32.1
Burkina Faso,30.310000000000002
Zambia,29.46
Ethiopia,29.080000000000005
South Sudan,28.729999999999997
Tanzania,28.39


## Find Countries Where The Death Rate Is Greater Than The Birth Rate

In [27]:
%%sql
SELECT name
FROM facts
WHERE death_rate > birth_rate;

Done.


name
Austria
Belarus
Bosnia and Herzegovina
Bulgaria
Croatia
Czech Republic
Estonia
Germany
Greece
Hungary
